### Sentiment Analysis

In [1]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()

hf_token = os.getenv('HF_TOKEN')
hf_auth = "Bearer " + hf_token

model = "cardiffnlp/twitter-roberta-base-sentiment-latest"

API_URL = f"https://api-inference.huggingface.co/models/{model}"
headers = {"Authorization": hf_auth}
payload = {
    "inputs": "Today is a great day",
}

response = requests.post(API_URL, headers=headers, json=payload)
response.json()

[[{'label': 'positive', 'score': 0.9836677312850952},
  {'label': 'neutral', 'score': 0.01135887298732996},
  {'label': 'negative', 'score': 0.004973393864929676}]]

In [3]:
from huggingface_hub import InferenceClient

client = InferenceClient(
    model,
    token=hf_token,
)

client.text_classification("Today is a great day")

[TextClassificationOutputElement(label='positive', score=0.9836677312850952),
 TextClassificationOutputElement(label='neutral', score=0.01135887298732996),
 TextClassificationOutputElement(label='negative', score=0.004973393864929676)]

### gpt 2 response

In [11]:
API_URL = "https://api-inference.huggingface.co/models/gpt2"
headers = {"Authorization": hf_auth}  # Replace with your valid token

# Define the query function to send a request to the Hugging Face API
def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

# Define the text generation function using the query function
def generate_text(prompt):
    response = query({"inputs": prompt})
    
    # Print the raw response to inspect its structure
    print(response)
    
    # Safely access the response, handling errors
    if isinstance(response, list) and len(response) > 0:
        print(response)
        return response[0].get('generated_text', "No text generated.")
    else:
        return "Error: Unable to generate text."

# Provide a text prompt
prompt = "Once upon a time in LA"
print(generate_text(prompt))

[{'generated_text': 'Once upon a time in LA, I must have cried a little bit. Our favorites were Lahart Taeya, who survived just after her divine verdict and what followed, and Joseph Kolokia, struggling with depression and being given up on starting a factory. She was smart, caring, innovative, and brave.\n\nAnd for us, he was not different. For a long time throughout his life, he took good care of things. Never brave, always sound, always off-putting. He joined the Navy many times, and went into Wall Street at one point accompanied by Randy Carlson, a smart business friend and female partner in the kind service to which I refer. We became pals and we went to compete together. It was definitely on the flip side of growth, ever since we were little. I remember going back to our car when he didn\'t return. He would say, "Hey, I need someone to step in here."\n\nOld time vs new time are a reflection of volunteerism and the civic responsibility that comes along with winning your city aroun

### Named Entity Recognition (NER) with BERT 

In [13]:
import time
import requests

API_URL = "https://api-inference.huggingface.co/models/dbmdz/bert-large-cased-finetuned-conll03-english"
headers = {"Authorization": hf_auth}  # Replace YOUR_API_KEY with your actual key

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

def ner(text):
    while True:
        response = query({"inputs": text})
        if 'error' in response and 'loading' in response['error']:
            print(response['error'])
            time.sleep(10)  # Wait for 10 seconds before retrying
        else:
            return response

text = "Hugging Face Inc. is based in New York City."
print(ner(text))

[{'entity_group': 'ORG', 'score': 0.9958662986755371, 'word': 'Hugging Face Inc', 'start': 0, 'end': 16}, {'entity_group': 'LOC', 'score': 0.9992396235466003, 'word': 'New York City', 'start': 30, 'end': 43}]
